In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.sparse
from scipy.sparse import *
from scipy.sparse import linalg
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import collections
import os, time, collections, shutil
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### loading data

In [3]:
names = ['squareId','timeInterval','countryCode','smsIn','smsOut','callIn','callOut','Internet']
dir_ = '../data/raw/milan/sms-call-internet-mi/sms-call-internet-mi-Nov/sms-call-internet-mi-2013-11-03.txt'
data = pd.read_table(dir_,names = names)
data = data.fillna(0)
d = data.groupby(['squareId','timeInterval']).sum()
d = d.drop(['countryCode'],axis = 1)
str_time = min(d.index.get_level_values(1))
end_time = max(d.index.get_level_values(1))

In [5]:
interaction =  load_npz('demo/1383464400000.npz')

In [9]:
inter = interaction.tocoo()

In [17]:
row = inter.row
col = inter.col
val = inter.data

In [21]:
edges = list(zip(row,col,val))

In [45]:
tmp = 'tmp.txt'
f = open(tmp, "a+")
for r,c,v in edges:
    f.write('{} {} {}\n'.format(r,c,v))
f.close()

### unsupervised embedding
进行简单的低维嵌入，只是简单的利用节点之间的通讯交互

#### deep walk


In [42]:
from libnrl.graph import *
from libnrl import node2vec

In [46]:
g = Graph()
g.read_edgelist(filename='tmp.txt', weighted=True, directed=True)

In [47]:
model = node2vec.Node2vec(graph=g, path_length=10,num_paths=5, dim=10,workers=4, window=5, dw=True)

Walk iteration:
1 / 5
2 / 5
3 / 5
4 / 5
5 / 5
Learning representation...


In [48]:
model.save_embeddings('embedding.txt')

#### forming edges